In [1]:
import chromadb

client = chromadb.PersistentClient(path="/Users/arad/repos/pp_kaggle_query/db")
collection = client.get_collection(name="kaggle")
collection.count()

30000

In [2]:
import openai
import chromadb.utils.embedding_functions as embedding_functions
from dotenv import load_dotenv,find_dotenv
import os

_ = load_dotenv(find_dotenv())
api_key = os.getenv("OPENAI_API_KEY")

openai.api_key = api_key

# from oai via chroma
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=api_key,
                model_name="text-embedding-3-small"
            )

In [4]:
# create user query collection
user_query_collection = client.get_or_create_collection(name="user",
    embedding_function=openai_ef, metadata={"hnsw:space": "cosine"})

3139007895

In [5]:
import uuid

# generate a random uuid
random_uuid = uuid.uuid4().int & (1<<32)-1

# convert the uuid to a 32-bit integer
random_id = random_uuid.int & (1<<32)-1
print(random_uuid)
print(random_id)

b3107436-5925-425a-9195-3e335513c7c1
1427359681


In [9]:
type(random_id)

int

In [14]:
collection.get(ids=['3607951', '3625760', '3636497', '3639234'],include=['embeddings','metadatas'])

# we need to make another "collection" for user queries
# when a user writes a query
## we add it to the user query collection (this will automatically embed it)
## we get the user query embedding out of that collection
## and pass it to the query function on the dataset description collection.


request = "I am looking for data on Indian crop yields"



def answer_users_request(request):
    query_id = uuid.uuid4().int & (1<<32)-1
    user_query_collection.add(ids=[str(query_id)], documents=[request])
    query_embedding = user_query_collection.get(ids=[str(query_id)],include=["embeddings"])['embeddings']
    
    query_response = \
    collection.query(
    query_embeddings=query_embedding,
    n_results=5
    # ,
    # where={"TotalDownloads": "is_equal_to_this"},
    # where_document={"$contains":"search_string"}
    )

    return query_response



In [15]:
telecoms_request = """ 

telecommunications usage. The dataset should include the usage date, provider ID, user ID, call duration, data usage, SMS count, service type (voice, data, SMS), location data, user demographics, and billing information. The dataset must be clean, with no missing values, consistent identifiers, accurate usage details, and provided in CSV format with clearly labeled columns and standardized date formats. Accompanying metadata describing data sources, preprocessing steps, and field definitions is required. An estimated delivery timeline and a mechanism for accessing updates, such as an API or regular data dumps, would be appreciated. The data will be used for usage pattern analysis and telecom service optimization, making accuracy and completeness critical.

"""
test_query = answer_users_request(telecoms_request)
print(test_query)

{'ids': [['4137128', '2082587', '2968687', '3137299', '3860469']], 'distances': [[0.34708380699157715, 0.35823768377304077, 0.36579829454421997, 0.3728839159011841, 0.39281564950942993]], 'metadatas': [[{'Title': 'Telecom dataset', 'TotalDownloads': 60, 'TotalUncompressedBytes': 0.0, 'TotalVotes': 4}, {'Subtitle': 'Really Big Telecommunication dataset', 'Title': 'Telecom IOT, Customer and Revenue Dataset', 'TotalDownloads': 1077, 'TotalUncompressedBytes': 0.0, 'TotalVotes': 16}, {'Subtitle': 'Telecom Company Service Performance Analysis', 'Title': 'Telecom Company Churn Rate, Call Center Data', 'TotalDownloads': 3043, 'TotalUncompressedBytes': 0.0, 'TotalVotes': 44}, {'Title': 'Telecom Campaign Managment', 'TotalDownloads': 29, 'TotalUncompressedBytes': 0.0, 'TotalVotes': 0}, {'Title': 'Telecom Customer Dataset', 'TotalDownloads': 17, 'TotalUncompressedBytes': 0.0, 'TotalVotes': 0}]], 'embeddings': None, 'documents': [["Title: Telecom dataset Subtitle: nan Description: ![](https://www.

In [18]:
for metadata in test_query['metadatas']:
    print(metadata[])


[{'Title': 'Telecom dataset', 'TotalDownloads': 60, 'TotalUncompressedBytes': 0.0, 'TotalVotes': 4}, {'Subtitle': 'Really Big Telecommunication dataset', 'Title': 'Telecom IOT, Customer and Revenue Dataset', 'TotalDownloads': 1077, 'TotalUncompressedBytes': 0.0, 'TotalVotes': 16}, {'Subtitle': 'Telecom Company Service Performance Analysis', 'Title': 'Telecom Company Churn Rate, Call Center Data', 'TotalDownloads': 3043, 'TotalUncompressedBytes': 0.0, 'TotalVotes': 44}, {'Title': 'Telecom Campaign Managment', 'TotalDownloads': 29, 'TotalUncompressedBytes': 0.0, 'TotalVotes': 0}, {'Title': 'Telecom Customer Dataset', 'TotalDownloads': 17, 'TotalUncompressedBytes': 0.0, 'TotalVotes': 0}]
